In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
train = pd.read_csv('/content/drive/My Drive/mnist-idx/mnist_train-1.csv')
test = pd.read_csv('/content/drive/My Drive/mnist-idx/mnist_test-1.csv')
#print(train.head())
x_train, x_validation, y_train, y_validation = train_test_split(train.iloc[:,1:],
    train.iloc[:,0],
    test_size=0.2,
    random_state=42,
    stratify=train.iloc[:,0])

In [4]:
# We'll create a "split" of training, for cross-validation
train_images_split = []
train_labels_split = []
validation_images = []
validation_labels = []

# We'll create a full training set
train_images_full = []
train_labels_full = []

# And the test set
test_images = []

for index, row in x_train.iterrows():
    train_images_split.append(row.values[0:].reshape((28, 28)))
    train_labels_split.append(row.iloc[0])
    train_images_full.append(row.values[0:].reshape((28, 28)))
    train_labels_full.append(row.iloc[0])

for index, row in x_validation.iterrows():
    validation_images.append(row.values[0:].reshape((28, 28)))
    validation_labels.append(row.iloc[0])
    train_images_full.append(row.values[0:].reshape((28, 28)))
    train_labels_full.append(row.iloc[0])

for index, row in test.iterrows():
    test_images.append(row.iloc[1:].values.reshape((28, 28)))

In [5]:
# Convert numpy array, while normalizing the image data
train_labels_split = np.array(train_labels_split)
train_images_split = np.array(train_images_split) / 255.
validation_labels = np.array(validation_labels)
validation_images = np.array(validation_images) / 255.
train_labels_full = np.array(train_labels_full)
train_images_full = np.array(train_images_full) / 255.
test_images = np.array(test_images) / 255.

print(train_labels_full.shape)
print(train_images_full.shape)
print(train_labels_split.shape)
print(train_images_split.shape)
print(validation_labels.shape)
print(validation_images.shape)
print(test_images.shape)

(59999,)
(59999, 28, 28)
(47999,)
(47999, 28, 28)
(12000,)
(12000, 28, 28)
(9999, 28, 28)


In [7]:
train_images = np.expand_dims(train_images_split, axis=3)
validation_images = np.expand_dims(validation_images, axis=3)

In [8]:
pre_trained_model =load_model('/content/drive/My Drive/small_train/checkpoint_accuracy281.hdf5')
pre_trained_model.trainable = False
#print(pre_trained_model.summary())
last_layer = pre_trained_model.get_layer('activation_11')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Model
# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# # Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# # Add a final sigmoid layer for classification
x = layers.Dense(10, activation='softmax')(x)
#
model = Model(pre_trained_model.input, x)

last layer output shape:  (None, 128)


In [10]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Generators
train_gen = ImageDataGenerator().flow(train_images, train_labels_split, batch_size=32)
validation_gen = ImageDataGenerator().flow(validation_images, validation_labels, batch_size=32)
checkpointer = keras.callbacks.ModelCheckpoint(filepath = r'/content/drive/My Drive/small_train/checkpoint-mnist-transfer-learning.hdf5',
                                               monitor='accuracy',
                                               mode='min',
                                               save_best_only=False,
                                               verbose=1)

In [12]:

# Model fit
history = model.fit(train_gen,
                    validation_data = validation_gen,
                    steps_per_epoch = len(train_images) / 32,
                    validation_steps = len(validation_images) / 32,
                    epochs=5,
                    callbacks = [checkpointer])

Epoch 1/5
1499/1499 [==============================] - 23s 16ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000

Epoch 00001: saving model to /content/drive/My Drive/small_train/checkpoint-mnist-transfer-learning.hdf5
Epoch 2/5
1499/1499 [==============================] - 23s 15ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000

Epoch 00002: saving model to /content/drive/My Drive/small_train/checkpoint-mnist-transfer-learning.hdf5
Epoch 3/5
1499/1499 [==============================] - 23s 15ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000

Epoch 00003: saving model to /content/drive/My Drive/small_train/checkpoint-mnist-transfer-learning.hdf5
Epoch 4/5
1499/1499 [==============================] - 23s 15ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000

Epoch 00004: saving model to /content/drive/My Drive/small_train/chec